# Exploration of World Co2 Emissions 
## Applications of distributions developed by Kuznets, Lorenz, and Gini 
### Python Course Final Project | Adam Russel Heisey | 04/27/2022

### STEP 1: IMPORT LIBRARIES, PACKAGES, AND FUNCTIONS FOR ANALYSIS

In [1]:
import pandas as pd
import numpy as np
from numpy import log as ln
import requests
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.api as sm
from numpy import log as ln
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.stats import pearsonr
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as MSE
from pandas.plotting import register_matplotlib_converters
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier 

#### -> Following the tutorial cited below, I adapted the following function to define downloading data from a url link, which allows me to download a csv file directly from a website link directly into my directory under a name I assign.

In [ ]:
def download_file(url, filename=' '):
    try:
        if filename:
            pass
        else:
            filename = req.url[downloadURL.rfind('/')+1:]

        with requests.get(url) as req:
            with open(filename, 'wb') as f:
                for chunk in req.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            return filename
    except Exception as e:
        print(e)
        return None
    
#works cited: tutorial on creating this code-->  https://learndataanalysis.org/create-a-python-program-to-download-file-from-the-web-python-tutorial/

### STEP 2: DATA SOURCE AND DOWNLOAD

#### -> Using Kaggle.com, I found a website called Our World in Data (link below), which has compiled datasets of information across numerous sectors. Because I am interested in exploring Co2 emissions around the world, I chose to download their complete Co2 dataset. The link to the dataset and the codebook are found at thier github page, which I sited below.

In [ ]:
x = 'https://nyc3.digitaloceanspaces.com/owid-public/data/co2/owid-co2-data.csv' #copied the url for the complete csv dataset and named the link address 'x'
download_file(x, 'co2.csv') #used the function above to download the csv file to my workspace folder and named the file 'co2.csv'

#works cited: co2 dataset github page -> https://github.com/owid/co2-data
#works cited: Our world in Data website-> https://ourworldindata.org/

#### -> Using the PANDAS package, I read the csv file and named the dataframe 'co' 

In [ ]:
co = pd.read_csv('co2.csv') #read csv file and named pandas dataframe 'co'
print(co.head(5)) #to explore the first 5 rows
print(co.info()) #to see a list of the columns, index, etc. 

In [ ]:
co.isnull().sum() #checking DF for sum of null values for each column

#### -> From above, I can see that this dataset has 60 variable columns, and 25989 rows that includes time-series data for most (not all) countries in the world. For panel data analysis the data would be most helpful to be organized by time ('year') and country ('iso code'). Also, I can see that there are a lot of missing values, which indicates an unbalanced/incomplete world dataset.Using PANDAS, I re-read the csv file and indexed the df by the column year. 

In [ ]:
co = pd.read_csv('co2.csv', parse_dates = ['year'], index_col = 'year') #parse dates pulls the year and puts it into the datetime helpful for indexing on year column. 
print(co.head(5)) #print first 5 rows
print(co.tail(5)) #print last 5 rows

### STEP 3: DEFINE OBJECTIVES
#### -> Based on the preliminary analysis of this data source, because there are a lot of missing values (years) for countries and because there are a lot of columns, I defined some goals/objectives:

##### 1) Reshape dataset and clean missing data for Panal Data analysis--multi level country (ID) by year for 2010 to 2019 (with the assumption that there are more data rows post 2010 and missing data for countries in 2020-either not published or missing bc COVID-)

##### 2) Run Fixed Effects Panel regression to test Kuznets Curve relationship--as countries develop(grow gdp) they initally produce more Co2, but decrease co2 emissions as they develop.

###### Simple model: co2 = gdp + gdp^2 + energy consumption + e

##### 3) Take begining and end (2010 and 2019) cross-sections and compare world inequality of Co2 emission with Gini coefficent/lorenz curves

##### 4) Use plots to create data visualizations

#### -> 1) Reshape dataset for Panel Data regression analysis

In [ ]:
co1 = co.loc[:,['iso_code','country', 'co2_per_capita','energy_per_capita', 'population', 'gdp' ]] # create new df called 'co1' from 'co' df with 6 colums

startDate = '2010-01-01' #define start date for time series
endDate = '2019-01-01' #define end date for time series--this is exclusive

# Create columns for year, month, day to help create date as datetype datatype
co1['Year'] = co1.index.astype(str).str[:4]
co1['Month'] = 1 
co1['Day'] = 1
co1['Date'] = pd.to_datetime(co1[['Year','Month','Day']])

#set index to Date and sort each counry by index, which is year. This should show each country in starting in 2010. 
co1.set_index('Date',drop=True,inplace=True)
co1.sort_index(inplace=True)

#Overwrite df 'co1' with new 'co1' based on the start date and end date from above. 
co1 = co1[startDate:endDate]
co1 = co1.iloc[:,:]
co1['Date'] = co1.index

print(co1.head(5)) #print first 5 rows
print(co1.tail(5)) #print last 5 rows


In [ ]:
#using the pandas multi index I structured the 'co1' df to ogranize the time series by iso country and year and named the columns ID and year
co1.index = pd.MultiIndex.from_arrays(co1[['iso_code', 'Date']].values.T, names=['ID', 'year'])

co1.sort_index(inplace=True) #sorts both indeces the by ascending (default) in years and countries--starting with 'A' and 2010 in my df

mi = co1.set_index(["iso_code", "Date"]) #created a new df for my panal analysis called 'mi' (multi-index). This pulls the columns and indexes first by country code then by year

print(mi.head(5))#print first 5 rows
print(mi.tail(5))#print last 5 rows


#works cited: Data formats and estimation for Panel Data -> https://bashtage.github.io/linearmodels/index.html


#### Missing data-> Bc my dataset has a lot of missing data, I decided to only analyze countries that had non-missing values for each of the columns. This results in an unbalanced dataset with countries able to be fall in and out depending on the availability of data for each year

In [ ]:
mi = mi.dropna() #drops rows with missing values. Limits by data to only include countries that have non-missing data for each column. 

# convert the variables from level to log form to allow for easier interpretation of results:
mi['gdp_per_capita'] = (mi['gdp']) / (mi['population']) #creates a new column based on existing columns --gdp per capita
mi['gpd_per_capita_squ'] = (mi['gdp_per_capita'])*(mi['gdp_per_capita']) #calculates and add new column log of gdp per capita squared
mi['ln_gdp_per_capita'] = ln(mi['gdp_per_capita']) #new column that is the log of gdp per capita
mi['ln_gpd_per_capita_squ'] = (mi['ln_gdp_per_capita'])*(mi['ln_gdp_per_capita']) #calculates and add new column log of gdp per capita squared
mi['ln_co2_per_capita'] = ln(mi['co2_per_capita']) #new column that is the log of co2 per capita
mi['ln_energy_per_capita'] = ln(mi['energy_per_capita']) #new column that is the log of energy per capita

print(mi.head(5)) #print first 5 rows
print(mi.tail(5)) #print last 5 rows

print(mi.info()) #prints info for panel df 'mi'

print(mi['country'].unique()) #prints the list of unique countries in my analysis
print(mi['country'].nunique()) #prints the number of unique countreis in my analysis

mi.to_csv('Co2Paneldataformat.csv') #saves df to csv
mi.to_stata('Co2Paneldataformat.dta') #saves df to stata data file

#### -> 2) Run Panel regression with FIXED EFFECTS (entity effects in python) with Robust standard errors to account for heteroskedacity

In [ ]:
Y = (mi.loc[:,['ln_co2_per_capita']]) #set my dependent variable--log of co2 emissions
Xs = sm.tools.tools.add_constant(mi.loc[:,['ln_energy_per_capita', 'ln_gdp_per_capita', 'ln_gpd_per_capita_squ']]) # my logged independent variables adding a constant term

# fixed effects model
model_fe = PanelOLS(Y, Xs, entity_effects = True) #add fixed effects to account for unobservable, unit specific, and time invariant effects
fe_res = model_fe.fit(cov_type = "robust") #add robust standard errors to correct for heteroskedacity
#print results
print(fe_res)

#works cited: help with panel regression in python -> https://towardsdatascience.com/a-guide-to-panel-data-regression-theoretics-and-implementation-with-python-4c84c5055cf8

# RESULTS:

#### From the regression results above, we can see that this model has statistically significant coefficents with p-values (0.0000) and and overall high between and overall R-squared (0.92), which suggest that this model accounts for 92% of the variance within panel units; making this a relatively good model. 

#### The results, with the postive coefficents for gdp and negative for gdp squared, support the inverted U-shape Kuznets curve hythopthesis as related to environmental degration and co2 emissions:  That co2 emissions increases with econonomic growth (increased gdp) then reach some optimal level, and then co2 emissions start to fall(as countries start to move away from manufacturing or start to regulate co2 emissions). Below is a graphical representation with a few outliers at the top, but you can almost visualize the inverted U-shape. I will attempt to show this relationship using some cross-sections of specific years. 

#### The coefficient on energy per capita suggest a 1% increase in energy consumption per capita results in a 0.76% increase in co2 emissions per capita, holding all else equal. 

In [ ]:
sb.regplot(x="gdp_per_capita", y="co2_per_capita", data=mi, x_estimator=np.mean, logx=True) #plots the regression of gdp per capita and co2 emissions per capita.

# Note: it is important to note, this is not a plot of the panel data residuals 

#### -The Theorectial Framework-

In [ ]:
# create 100 equally spaced points between -10 and 10
x = np.linspace(-10, 10, 100)

# Basic theoretical relationship--inverse U
y = x - x**2 

with plt.style.context('seaborn-dark-palette'):
    plt.plot(x, y)
    plt.xlabel("ECONOMIC GROWTH (gdp per capita")
    plt.ylabel("ENVIRONMENTAL POLLUTION (co2 per capita)")
    plt.title("ENVIRONMENTAL KUZNETS CURVE")

#### -> 3) Estimating Gini coefficents for Co2 inequality with Lorenz curves

In [ ]:
mi['Year'].unique() #check to see how many complete years I have. 

In [ ]:
#collect cross-sections of co2 data for 2010 and save df as 'coy10'
coy10 = mi.loc[(mi['Year'] == '2010')]
coy10 = coy10.loc[:,['country', 'co2_per_capita', 'gdp_per_capita']]
#coy10

#collect cross-sections of co2 data for 2018 and save df as 'coy18'
coy18 = mi.loc[(mi['Year'] == '2018')]
coy18 = coy18.loc[:,['country', 'co2_per_capita', 'gdp_per_capita']]
#coy18



#### -> Following the link cited below, I adapted the following function to calculate the Gini Coefficent, which is a measure of statistical dispersion intended to represent inequality of distributions in a population group. I adapted its use, which is usually applied to wealth or income, to represent the inequality of co2 emissions across 165 countries in the world in 2010 and the same countries in 2018. I found several authors/adaptations of the gini coeffiecnt, but the one below allows for easy graphing/visualizations because it not only calculates the Gini coefficient, but it also returns the differnent areas under the curve, which makes plotting the results more efficient. 

#### -> Here is a link to more information about the Gini Coefficient: https://en.wikipedia.org/wiki/Gini_coefficient



In [ ]:
def G(v):
    bins = np.linspace(0., 100., 10)
    total = float(np.sum(v))
    yvals = []
    for b in bins:
        bin_vals = v[v <= np.percentile(v, b)]
        bin_fraction = (np.sum(bin_vals) / total) * 100.0
        yvals.append(bin_fraction)
    # perfect equality area
    pe_area = np.trapz(bins, x=bins)
    # lorenz area
    lorenz_area = np.trapz(yvals, x=bins)
    gini_val = (pe_area - lorenz_area) / float(pe_area)
    return bins, yvals, gini_val

#Works cited: Gini Coefficent assistance (author unknown) -> https://stackoverflow.com/questions/39512260/calculating-gini-coefficient-in-python-numpy

In [ ]:
v = coy10['co2_per_capita'].values #returns the vector values of co2 emissions in the 2010 cross section
z = coy18['co2_per_capita'].values #returns the vector values of co2 emissions in the 2018 cross section
vbins, vresult, vgini_val = G(v) #calculates the Gini for 2010 vector
zbins, zresult, zgini_val = G(z) #calculates the Gini for 2018 vector
with plt.style.context('seaborn-dark-palette'): #changed the style, just for fun
    plt.figure()
    plt.plot(vbins, vresult, label="2010 GINI: %.4f" %(vgini_val)) #plots the results for 2010
    plt.plot(vbins, vbins, '--', label="perfect equality") #draws teh perfect equality line
    plt.plot(zbins, zresult, label="2018 GINI: %.4f" %(zgini_val)) #overlays and plots the results for 2018

    #below I added axis and graph titles/labesl
    plt.xlabel("Fraction of sample population")
    plt.ylabel("Fraction of Co2 Emissions")
    plt.title("LORENZ CURVES OF CO2 EMISSIONS DISTRIBUTIONS 2010 VS 2018")
    plt.title("LORENZ CURVES OF CO2 EMISSIONS DISTRIBUTIONS 2010 VS 2018")


    plt.legend()
plt.show()

#added histogram of co2 emissions to get the counts of countries in the co2 per capita bins (10)
with plt.style.context('seaborn-dark-palette'): #changed the style, just for fun
    plt.figure()
    plt.subplot(2, 1, 1)
    plt.hist(v, bins=10, label=True)
    plt.subplot(2, 1, 2)
    plt.hist(z, bins=10)
    plt.xlabel("Co2 emission per Capita 2010(top) vs. 2018(bottom)")
    plt.ylabel("Number of Countries")

plt.show()

# Results

#### From the Lorenz curves, and the Gini coefficients calcualted/represented above, we can see that for the population of 165 countries in my sample, the distribution of C02 emissions has decreased by  ~2.17 percentage points, which is a 3.75% decrease in the inequality of Co2 emissions across the 165 countries. Perfect equality is represted with a Gini of 0 and Perfect Inequality with a Gini of 1. Although co2 emissions is becoming more equal, a Gini greater than .50 still represents high inequality, which means that there are higher levels of co2 emissions per capita by a fewer number of countries.This is represented in the histogram above; We can see that there is a higher proportion of countries in the the lower levels of co2 emissions per capital bins. 

#### -> 4) Apply EDA, regression, and plots to cross-section of dataset for 2018

In [ ]:
#create new dataframe for 165 countries in 2018
a18 = mi.loc[(mi['Year'] == '2018')] 

#show statistical information for cross secton df 'a18'
a18.describe()

sorted_a18 = a18.sort_values(by ='energy_per_capita')

#### First, I want to see how good my model is at predicting Co2 levels using linear regression

In [ ]:
# Create feature and target arrays
y = a18['ln_co2_per_capita'].values
X = a18[['ln_energy_per_capita', 'ln_gdp_per_capita', 'ln_gpd_per_capita_squ']].values

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

# Create the regressor: reg_all
reg_all = LinearRegression()

# Fit the regressor to the training data
tres = reg_all.fit(X_train,y_train)

# Predict on the test data: y_pred
y_pred = reg_all.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2: {:.2f}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error: {:.2f}".format(rmse))
print((reg_all.coef_))
print(reg_all.intercept_)

#### Next, I want to apply my linear regression to the entire sample, save the predictions, and graph the predictions vs. GDP per capita

In [ ]:
# Create feature and target arrays
y = a18['ln_co2_per_capita'].values
X = a18[['ln_energy_per_capita', 'ln_gdp_per_capita', 'ln_gpd_per_capita_squ']].values

# Create the regressor: reg_all
reg_all = LinearRegression()

# Fit the regressor to the entire series
tres = reg_all.fit(X,y)

# Predict on the entire dataset: y_pred
y_predall = reg_all.predict(X)

#save the predictions as a df called 'prediction_df'
prediction_df = pd.DataFrame(data=y_predall, index=None, columns=['predicted_co2']) 

#create a df called 'gdpk_df', which only includes gdp_per_capita
gdpk_df = pd.DataFrame(data=a18, index=None, columns=['gdp_per_capita']) 

#reset the index for 'gdpk_df' 
gdpk_df=gdpk_df.reset_index()

#merge the predictions df and gdpk_df using the index--call it 'df'
df = gdpk_df.join(prediction_df)

#print the first 5 rows of the new df 'df'
print(df.head(5))

#### To visualize the Kuznets curve in my data, I graphed the predicted co2 emissions vs GDP per capita. We can start to visualize the statisitically significant quadratic relationship we established in the panel regression: that as countries grow and develop, they actually start to decrease their co2 emissions. 

In [ ]:
sb.jointplot(x='gdp_per_capita', y='predicted_co2', data=df)

In [ ]:
mi

In [ ]:
#create new df for time-series graphs--reset the index from multi level to just date
new = mi.reset_index()
new.set_index('Date',drop=True,inplace=True)
print(new.tail(2)) #show first 2 rows

In [ ]:
#Sort 2018 df 'a18' by energy consumption per capita--default ascending
sorted_a18 = a18.sort_values(by ='energy_per_capita')

#Identify the TOP 6 energy consumers per capita in 2018
print(sorted_a18.tail(6))

In [ ]:
#create df for top 6 energy consumers in 2018 and plot the time series of co2 per capita from 2010-2018
cd1 = 'ARE'
cd2 = 'BHR'
cd3 = 'TTO'
cd4 = 'SGP'
cd5 = 'QAT'
cd6 = 'ISL'

#create country specific df
c1 = new[new['iso_code']== cd1]
c2 = new[new['iso_code']== cd2]
c3 = new[new['iso_code']== cd3]
c4 = new[new['iso_code']== cd4]
c5 = new[new['iso_code']== cd5]
c6 = new[new['iso_code']== cd6]

#plot each of the country time series
with plt.style.context('seaborn'): #changed the style, just for fun
    plt.plot(c1['co2_per_capita'], label = cd1 )
    plt.plot(c2['co2_per_capita'], label = cd2)
    plt.plot(c3['co2_per_capita'], label = cd3)
    plt.plot(c4['co2_per_capita'], label = cd4)
    plt.plot(c5['co2_per_capita'], label = cd5)
    plt.plot(c6['co2_per_capita'], label = cd6)
    plt.legend()
    plt.title('Co2 per Capita for TOP 6 from 2010-2018')
    plt.ylabel('Co2 per Capita (metric tons)')
plt.show()

In [ ]:
#Identify the lowest 6 energy consumers per capita in 2018
print(sorted_a18.head(6))

In [ ]:
#create df for lowest 6 energy consumers in 2018 and plot the time series of co2 per capita from 2010-2018
cd1 = 'BDI'
cd2 = 'CAF'
cd3 = 'NER'
cd4 = 'TCD'
cd5 = 'COD'
cd6 = 'RWA'

#create country specific df using the country codes above so I only have to type them once
c1 = new[new['iso_code']== cd1]
c2 = new[new['iso_code']== cd2]
c3 = new[new['iso_code']== cd3]
c4 = new[new['iso_code']== cd4]
c5 = new[new['iso_code']== cd5]
c6 = new[new['iso_code']== cd6]

#plot the time seris of co2 emissions for the lower 6 energy users in 2018
with plt.style.context('seaborn'): #changed the style, just for fun
    plt.plot(c1['co2_per_capita'], label = cd1 )
    plt.plot(c2['co2_per_capita'], label = cd2)
    plt.plot(c3['co2_per_capita'], label = cd3)
    plt.plot(c4['co2_per_capita'], label = cd4)
    plt.plot(c5['co2_per_capita'], label = cd5)
    plt.plot(c6['co2_per_capita'], label = cd6)

    plt.legend()
    plt.title('Co2 per Capita for LOW 6 from 2010-2018')
    plt.ylabel('Co2 per Capita (metric tons)')
plt.show()

In [ ]:
#split Burunid data from 2010-2018 into two parts
first = c1[c1['Year'] <= '2014']
second = c1[c1['Year'] > '2014']



In [ ]:
#conduct a T test to see if the means of two samples are the same--Null: sample1 = sample2
stats.ttest_ind(first['co2_per_capita'], second['co2_per_capita'])

###### ^fail to reject at the 0.5 level, indicating the two series are not statisically differnt from each other

In [ ]:
#check to see how many countreis are in my datset
print(new.groupby('iso_code').ngroups)

In [ ]:
#return the coutry codes keys
print(new.groupby('iso_code').groups.keys())

#### Peform a test using groupby to see if the co2 per capita are equivalent in usa and canada

In [ ]:
#assign the values of co2 emission per capita for the United States and canada using groupby
usa = new.groupby('iso_code').get_group('USA')['co2_per_capita']
can = new.groupby('iso_code').get_group('CAN')['co2_per_capita']

#test the null that the two series are equal usa=can
stats.ttest_ind(usa, can)

###### ^reject the null at  the 0.05 level, conclude that the two series are statistically differnt from one another

#### Use the KNeighborsClassifier to Perform a test to see if I gdp and energy consumption can predict being above the average c02 emission

In [ ]:
# using the 2018 cross section data I calculated the mean co2 emissions
print(a18['co2_per_capita'].mean())

In [ ]:
#created a dummy variale for countries that are emitting above the average for all countries---1 if above 0 if below 
a18.loc[:,'Above'] = (a18.loc[:, 'co2_per_capita'] > 4.74).astype(np.float64)


In [ ]:
# Creating arrays
y = a18['Above'].values
X = a18[['ln_energy_per_capita', 'ln_gdp_per_capita']].values

In [ ]:
# Create a k-NN classifier with 2 neighbors: knn
knn = KNeighborsClassifier(n_neighbors=2)

# Fit the classifier to the data
knn.fit(X,y)

# Predict the labels for the training data X
y_pred = knn.predict(X)



In [ ]:
print('Percent of countries above the average Co2 emission per capita: ' , np.mean(y)*100, '%')
print('Percent of countries predicted to be above the average Co2 emission per capita: ' , np.mean(y_pred)*100, '%')

In [ ]:

# Creating arrays
y = a18['Above'].values
X = a18[['ln_energy_per_capita', 'ln_gdp_per_capita']].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify=y)

# Create a k-NN classifier with 3 neighbors: knn
knn = KNeighborsClassifier(n_neighbors=3)

# Fit the classifier to the training data
knn.fit(X_train,y_train)

# Predict using the classifier
y_pred = knn.predict(X_test)

# Print the accuracy
print('accuracy')
print("Accuracy: {:.2f}".format(knn.score(X_test, y_test)))
print('-----------------------------------')
#print the confusion matrix
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred, labels=None, sample_weight=None, normalize=None))
print('upper left: true positive, lower left: false positive, upper right: false negative, bottom right: true negative')
print('------------------------------------')
print('classification report')
print('------------------------------------')
print(classification_report(y_test, y_pred))

In [ ]:
print('Fin')